In [ ]:
!pip install --upgrade google-cloud-storage
!pip install --upgrade PyPDF2
!pip install --upgrade six
!pip install --upgrade reportlab
!pip install --upgrade pandas
!pip install --google-auth


## 設定上傳檔案用的 Service Account （請修改檔案路徑）

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='[請填 Service Account 路徑]'

## 指定要使用的完課證明範本 （取消註解要使用的範本並註解其他範本 P.S 註解方式請在程式碼前加 # 字號）

In [ ]:
# Template 位置
template_file_location="templates/Completion_Template_BigData.pdf"
# template_file_location="templates/Completion_Template_Core_Infra.pdf"
# template_file_location="templates/Completion Template_GKE_3_day.pdf"
# template_file_location="templates/Completion Template_GKE_1_day.pdf"
# template_file_location="templates/Completion Template_Text_GenAI.pdf"


## PDF 建立程式碼 （除非要修改PDF Admin 密碼無需修改程式碼）

In [ ]:
# Font Source:https://github.com/google/fonts/tree/main/ofl

from PyPDF2 import PdfWriter, PdfReader
import io 
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

#請在此輸入 Owner PDF  密碼
password="[請輸入Owner 密碼]"

def gen_pdf(name, no_space_name, date, useremail):
    reportlab.rl_config.TTFSearchPath.append('fonts')
    pdfmetrics.registerFont(TTFont('NotoSansTC', 'NotoSansTC.ttf'))
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    can.setFont('NotoSansTC', 45)
    can.drawString(130, 400, name)
    
    can.setFont('NotoSansTC', 12)
    can.drawString(101, 197, date)
    can.save()

    #move to the beginning of the StringIO buffer
    packet.seek(0)

    # create a new PDF with Reportlab
    new_pdf = PdfReader(packet)
    # read your existing PDF
    existing_pdf = PdfReader(open(template_file_location, "rb"))
    output = PdfWriter()
    # add the "watermark" (which is the new pdf) on the existing page
    page = existing_pdf.pages[0]
    page.merge_page(new_pdf.pages[0])
    output.add_page(page)
    output.add_metadata(
        {
            "/Author": "CloudMile",
            "/Producer": "CloudMile",
        }
    )
    output.encrypt(user_password=useremail,owner_password=password,permissions_flag=-3900)
    # finally, write "output" to a real file
    output_stream = open(f"tmp/{no_space_name}.pdf", "wb")
    output.write(output_stream)
    output_stream.close()

## 上傳 GCS 程式碼 （無需修改程式碼）

In [ ]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = None

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )



## GCS 建立 Singed URL （無需修改程式碼）

In [ ]:
import binascii
import collections
import datetime
import hashlib
import sys

# pip install google-auth
from google.oauth2 import service_account
# pip install six
import six
from six.moves.urllib.parse import quote


def generate_signed_url(service_account_file, bucket_name, object_name,
                        subresource=None, expiration=604800, http_method='GET',
                        query_parameters=None, headers=None):

    if expiration > 604800:
        print('Expiration Time can\'t be longer than 604800 seconds (7 days).')
        sys.exit(1)

    escaped_object_name = quote(six.ensure_binary(object_name), safe=b'/~')
    canonical_uri = '/{}'.format(escaped_object_name)

    datetime_now = datetime.datetime.now(tz=datetime.timezone.utc)
    request_timestamp = datetime_now.strftime('%Y%m%dT%H%M%SZ')
    datestamp = datetime_now.strftime('%Y%m%d')

    google_credentials = service_account.Credentials.from_service_account_file(
        service_account_file)
    client_email = google_credentials.service_account_email
    credential_scope = '{}/auto/storage/goog4_request'.format(datestamp)
    credential = '{}/{}'.format(client_email, credential_scope)

    if headers is None:
        headers = dict()
    host = '{}.storage.googleapis.com'.format(bucket_name)
    headers['host'] = host

    canonical_headers = ''
    ordered_headers = collections.OrderedDict(sorted(headers.items()))
    for k, v in ordered_headers.items():
        lower_k = str(k).lower()
        strip_v = str(v).lower()
        canonical_headers += '{}:{}\n'.format(lower_k, strip_v)

    signed_headers = ''
    for k, _ in ordered_headers.items():
        lower_k = str(k).lower()
        signed_headers += '{};'.format(lower_k)
    signed_headers = signed_headers[:-1]  # remove trailing ';'

    if query_parameters is None:
        query_parameters = dict()
    query_parameters['X-Goog-Algorithm'] = 'GOOG4-RSA-SHA256'
    query_parameters['X-Goog-Credential'] = credential
    query_parameters['X-Goog-Date'] = request_timestamp
    query_parameters['X-Goog-Expires'] = expiration
    query_parameters['X-Goog-SignedHeaders'] = signed_headers
    if subresource:
        query_parameters[subresource] = ''

    canonical_query_string = ''
    ordered_query_parameters = collections.OrderedDict(
        sorted(query_parameters.items()))
    for k, v in ordered_query_parameters.items():
        encoded_k = quote(str(k), safe='')
        encoded_v = quote(str(v), safe='')
        canonical_query_string += '{}={}&'.format(encoded_k, encoded_v)
    canonical_query_string = canonical_query_string[:-1]  # remove trailing '&'

    canonical_request = '\n'.join([http_method,
                                   canonical_uri,
                                   canonical_query_string,
                                   canonical_headers,
                                   signed_headers,
                                   'UNSIGNED-PAYLOAD'])

    canonical_request_hash = hashlib.sha256(
        canonical_request.encode()).hexdigest()

    string_to_sign = '\n'.join(['GOOG4-RSA-SHA256',
                                request_timestamp,
                                credential_scope,
                                canonical_request_hash])

    # signer.sign() signs using RSA-SHA256 with PKCS1v15 padding
    signature = binascii.hexlify(
        google_credentials.signer.sign(string_to_sign)
    ).decode()

    scheme_and_host = '{}://{}'.format('https', host)
    signed_url = '{}{}?{}&x-goog-signature={}'.format(
        scheme_and_host, canonical_uri, canonical_query_string, signature)

    return signed_url

## 寄信程式碼 （除非寄信設定需要修改，無需修改此程式碼）

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pathlib import Path
import smtplib

from string import Template

mail_address="[請輸入 Email Address]"
mail_pass="[請輸入 Application Password]"

def send_mail(to, subject, singned_url):
    content = MIMEMultipart()  #建立MIMEMultipart物件
    content["subject"] = subject  #郵件標題
    content["from"] = mail_address  #寄件者
    content["to"] = to #收件者


    # Mail Content from template
    template = Template(Path("email_body.html").read_text())
    body = template.substitute({ "link_address": singned_url })
    content.attach(MIMEText(body, "html"))

    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:  # 設定SMTP伺服器
        try:
            smtp.ehlo()  # 驗證SMTP伺服器
            smtp.starttls()  # 建立加密傳輸
            smtp.login(mail_address, mail_pass)  # 登入寄件者gmail
            smtp.send_message(content)  # 寄送郵件
            print(f"Mail successfully send to {to}")
        except Exception as e:
            print("Error message: ", e)

## 主程式與邏輯 （此程式碼會觸發學員完課證明製作並且將檔案上傳到 Cloud Storage，請依照前幾行程式碼要求修改參數）

In [ ]:
import pandas as pd
import reportlab

file_location='[請輸入目前此python檔案的檔案路徑]'
service_account_file='[請填 Service Account 路徑]'
bucket_name='[請填專案中 GCS 的 Bucket 名稱]'
# GCS File 夾名稱
course_info='[請填一個用於辨認此次課程的 GCS Folder 名稱]'
# 完課名單來源
complete_file_name="[請填入來源 CSV 檔案位置]"
# 完課日期
course_date="[ex: 2024/05/13]"



##以下請不要做任何修改
df = pd.read_csv(complete_file_name)
singed_url_list=[]


for i, data in df.iterrows():
    name= data["Name"]
    email= data["Email"]
    # Setup filename for pdf
    no_space_name=name.replace(" ","_")
    
    # Generate PDF with desired name 
    gen_pdf(name, no_space_name, course_date, email)
    
    # Setup where to get file from to upload to GCS
    source_file_location=f"{file_location}/tmp/{no_space_name}.pdf"
    
    # Setup GCS filename
    object_name=f"{course_info}/{no_space_name}.pdf"
    
    # Upload to GCS
    upload_blob(bucket_name, source_file_location, object_name)
    
    # Generate singed url for the pdf file
    singed_url=generate_signed_url(service_account_file, bucket_name, object_name,
                        expiration=604800, http_method='GET')
    
    # Add the singed url to a list for later usage
    if singed_url!=None:
        singed_url_list.append(singed_url)
    else:
        singed_urllist.appned("unknown error")

# Generate MailMerge CSV file
mailmerge_filename=f"{course_info}_mail_merge_list.csv"

df_new = pd.DataFrame()     
df_new["Name"]=df["Name"]    
df_new["Email"]= df["Email"]
df_new["singed_url"] = singed_url_list
df_new.to_csv(mailmerge_filename)

# Upload MailMerge file to GCS to put with other certificates
upload_blob(bucket_name, mailmerge_filename, f"{course_info}/{mailmerge_filename}")



## 請透過執行以下程式碼檢視即將寄信之名單以及相關檔案，若人數過多也可以到程式碼資料夾中直接尋找 mailmerge 檔案

In [ ]:
df_new

## 此程式碼會將信件寄出給學員，運行時程式碼會詢問發證人數以及 Double Check，請確實完成檢查再回答  Yes 寄信

In [ ]:
original_completetion = int(input(f"請輸入原始完課清單上有完課的人數"))
if(original_completetion!=len(df_new.axes[0])):
    print("人數與即將寄信之人數不一致！！！請檢查")
else:
    confirmation = input(f"你即將寄信給客戶，請確認即將發放證明之人數{len(df_new.axes[0])}人、姓名、GCS 清單，確認請輸入yes信件就發出: ")
    if(confirmation=="yes"):
        # Send Email
        for i, data in df_new.iterrows():
            send_mail(data["Email"], "[CloudMile][No Reply] Congratulations on completing your course!", data["singed_url"])
    else:
        print("You did not confirm the message. Your email won't be sent")